# Druid Schema Demo
Druid is a wonderful system for high-performance queries of event data.
Proper tuning is essential to obtain optimal performance.
However, it can be a bit awkward to visualize parts of the system.
This demo shows how we can use the [Druid REST API](https://druid.apache.org/docs/latest/operations/api-reference.html), Python and [Jupyter](https://jupyter.org/)
to fill in gaps in the Druid UI.

This demo shows how we can obtain the schema for a table, in our case,
the [demo Wikipedia data table](https://druid.apache.org/docs/latest/tutorials/index.html)
running in a [Druid hosted on Docker](https://druid.apache.org/docs/latest/tutorials/docker.html) on
the local machine. We then show how we can analyze that schema to look for
cost and performance optimizations.

You don't heed a live Druid however, to following this demo,
because [Jupyter Notebook Viewer](https://nbviewer.jupyter.org/) 
is showing you an already-completed notebook.

The demo is based on two Python libraries: [pydruid](https://pypi.org/project/pydruid/) for queries, and a bespoke
library, in this project, for other APIs and to prepare the reports.

We start with some Python basics.

In [1]:
import api
import report
import model
report.styles()

Next we create a client for our endpoint and create an instance of the
`Report` class that will prepare the reports. (Druid's router was placed
on port 8899 so it does not conflict with Jupyter on port 8888.)

In [2]:
config = api.default_local_docker_config(8899)
client = api.Druid(config)
reports = report.Reports(client)

Let's first see what data sources we've defined. There are two versions
of Wikipedia:

In [3]:
reports.tables()

Table Name
wikipedia
wikiticker-2015-09-12-sampled


Now, let's explore the schema of that second table:

In [4]:
table = reports.table('wikiticker-2015-09-12-sampled')
table.schema()

## Analysis
From here we can start our analysis of this table. We'll focus on the schema
for now, leaving the storage optimization for another demo.

A key question to ask is: are the columns pulling their own weight? Druid is
an in-memory database&mdash;we want to ensure we use our memory wisely. A
good place to start is with the largest columns as given by "Row %": the
percentage of the entire row width taken up (on average) by the given
column.

In Druid, `string` columns are compressed using dictionary encoding. So, the
most efficient storage is if we have all columns with the same value, even if
that value is a hundred bytes long. The most inefficient is when each row has
a distinct value: then the dictionary does not provide any compression and just
acts as overhead.

In our example, the `comment` column stands out: it takes 31% of the row and
contains 1285 distinct values. (This is somewhat surprising, it says that only
3% of edits bothered to explain their changes; assuming unique comments.)

To decide if we can remove this dimension, we'd need to know if it is used,
which is the topic of a later demo. For now, we can simply examine our goals.
Unless our goal involves textual analysis of comments (a job for which Druid is
not optimized), we really don't need this column. If we drop it, we save
2 MB out of 6.4 MB, reducing our table size to 4.4 MB. Pretty good savings
for little work. If this had been real data, the savings could be substantial.

The next largest column is `page`, which is the name of the page. Here the
decision is harder: it seems likely that someone will want statistics by page
(to find, for example, the top-10 active pages). So we probably need to keep
this one.

It is interesting to note the number of dimensions which are *not* problems.
For example, we might wonder if anyone uses `regionIsoCode`. But, since it
consumes only 0.1% of the table space, we might as well keep it.

Also, the [documentation](https://druid.apache.org/docs/latest/design/segments.html#sql-compatible-null-handling)
says that nullable columns (those that allow SQL
`NULL` values) have an overhead. But, that overhead must be pretty small as
shown by the tiny column sizes for nullable columns such as `regionIsoCode`
(0.1%) and `regionName` (0.4%).

In summary, once you see the amount of precious memory space used by a dimension,
you can start to make informed decisions about what to keep and what to drop.

### Next Steps

From here, we'd like to explore other aspects of our table to look for other performance opportunities:

* Which columns are actually used in queries? Are some unused so we can remove them?
* Which time grain is used in queries? Should we store the data at a larger time grain (i.e. use rollups?)
* How many segments are we creating? Are we creating the [optimal segment size](https://druid.apache.org/docs/0.20.0/operations/basic-cluster-tuning.html#segment-sizes-matter)? How would we [optimize](https://druid.apache.org/docs/0.20.0/operations/segment-optimization.html) the size?
* Which segments are queries hitting? Should we set [data expiration](https://druid.apache.org/docs/0.20.0/ingestion/data-management.html#retention) to occur sooner?
* How much data will we end up storing over a day, week or lifetime (considering retention rules)?
* We've talked about improvements in terms of memory size. What does this translate to in terms of dollar amounts for our given servers?

We can expand the techniques used here to answer some of these questions. Those will be the topic of future demos.

## Notebook Details
This section explains how the Notebook does its thing, in case you want to do something
similar. The `api.py` module is a wrapper around a subset of the Druid API: just enough
to do the analysis here. It would be trivial to extend this to other APIs (contributions
welcome.)

The `Druid` class needs only the Router endpoint: host and port. It uses a REST call to
learn about the other nodes of the cluster, including the Coordinator. In a Docker
environment, the host names (actualy IP addresses) which Druid reports are not what the
client sees, so the `default_local_docker_config()` function sets up a mapping between
the node addresses and the host running Docker (in this case, `localhost`.) Ports are
assumed to be exposed on the same port number, though it would be simple to add port mapping
to the configuration. The code does handle port mapping for the Router since the default
Druid port of 8888 happens to the the same as the default Jupyter port.

The code organizes operations by node kind (Router, Coordinator, etc.). Then, since we
often work with datasources, and datasource operations are spread across nodes, we also
have a `DataSource` class which acts as a facade on top of the node-level APIs.

In Druid, there is no one API call to get table metadata. So, a `TableModel` class uses
multiple APIs to gather the data. One of these, the [metadata query](https://druid.apache.org/docs/latest/querying/datasourcemetadataquery.html) (via [pydruid](https://pypi.org/project/pydruid/)) is costly
on a large system, so getting the information might take a while.

Finally, there is a `Report` class that creates the reports as HTML tables. If you are
familiar with [Pandas](https://pandas.pydata.org/), you could easily load the data 
into a Pandas dataframe. That as
not done here in the interest of minimizing dependencies.

## Technical Comments
Finally, this exercise suggested a few possible Druid improvements.

### Schema Tab in Druid UI

The most obvious suggestion is simply to add table schema information (such as that illustrated above) to the Druid UI. From the Datasources tab, I should be able to click on a table to see the metadata. (At present, clicking on a table simply filters the view to only show that one table.)

### Long Column Compression

Consider the columns `added`, `deleted` and `delta` in the table above. Each represents the number of lines(?) changed by an edit. Such numbers will tend to skew quite small. Yet, according to the table above, Druid dedicates 8 bytes per row for these values: the full length of a `long`.

As it turns out, [Protobuf](https://developers.google.com/protocol-buffers/docs/encoding) provides functions that compresses integers down to the number of bits actually needed, which can be very useful in an columnar encoding.
[Parquet](https://github.com/apache/parquet-format/blob/master/Encodings.md) does something similar.

Our example table has only three `long` fields, so the savings here would not be large, perhaps 18 bytes per row (or 10%). However, in a heavily-numeric row, the savings could be substantial.

### Boolean Columns

Consider the various Boolean (true/false) values in the Wikipedia dataset, such as `isAnonymous`. Each takes about 4 bytes per row to store. The data load was naive: it used the default parsing which treated the fields as strings with values "true" and "false". One observation is that, if a string column has low cardinality, we don't need four bytes to store the dictionary location. Specifically, the dictionary index could be encoded as suggested above for `long` values.

Since these are Boolean columns, and SQL has support for Boolean types such as `if(isAnonymous, ...)` or `ISTRUE(isAnonymous)`, it would be handy to have a Boolean type. Since the cardinality of  Boolean type is 2, each column would require just one bit. Or, if it is more convenient to use a byte, then Protobuf or Parquet-style compression would reduce that bit down to a bit (or less, for repeated values) per column.

In the example table above, we have five Boolean columns, so the savings would be around 15 bytes per row, or 8% of the row size. (Or 11%, once we remove the wasteful `comment` column.) This savings could be interesting, without the user having to play any tricks.

### Rationalize Metadata REST Calls

Creating this Notebook as a bit more painful than seemed necessary. It would certainly have been helpful to have a complete Python client, but it is not hard to knock one out. Rather, the most painful aspect was that many of Druild's [metadata APIs](https://druid.apache.org/docs/0.20.0/operations/api-reference.html#metadata-store-information) (also [here](https://druid.apache.org/docs/0.20.0/operations/api-reference.html#datasource-information) and [here](https://druid.apache.org/docs/0.20.0/operations/api-reference.html#get-22), among others) either don't work at all, or don't do what the docs say. Some amount of fumbling around was needed to find a set that gave the desired data (though no one API provides the whole picture.) Clearly there has been some volution over time in what the APIs do. It would be very helpful to users to mark the APIs which are obsolete and to instead point the user to the preferred methods.